# Graph neural networks - Practicals 2

### Installation of PyTorch

In [1]:
import torch
TORCH = torch.__version__.split('+')[0]
DEVICE = 'cu' + torch.version.cuda.replace('.', '') if torch.cuda.is_available() else 'cpu'

!pip install torch-sparse torch-scatter torch-cluster torch-spline-conv -f https://pytorch-geometric.com/whl/torch-{TORCH}+{DEVICE}.html
!pip install torch-geometric
!pip install networkx==3.1 matplotlib tqdm torchmetrics ipywidgets

Looking in links: https://pytorch-geometric.com/whl/torch-2.0.1+cpu.html


In [2]:
import torch.nn as nn
import torch.nn.functional as F
import torchmetrics
import torch_geometric.nn as gnn
import torch_geometric as tg
from torch_geometric.data import Dataset
from torch_geometric.loader import DataLoader
import networkx as nx
import matplotlib.pyplot as plt
from typing import Tuple
# from tqdm.notebook import tqdm
from tqdm import tqdm


def train_and_eval(model : nn.Module, dataset : Dataset) -> Tuple[nn.Module, float]:
    """
    Train and evaluate the model on the dataset.
    A boilerplate for training and evaluating a neural network.

    Args:
        model (nn.Module): The model to train and evaluate.
        dataset (Dataset): The dataset to train and evaluate on.

    Returns:
        nn.Module: The trained model with the best validation accuracy.
        float: The best validation accuracy.
    """

    # init training and validation dataloaders
    dataloader_train = DataLoader(dataset[:150], batch_size=16, shuffle=True)
    dataloader_val = DataLoader(dataset[150:], batch_size=16, shuffle=False)

    # init optimizer and loss
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    criterion = nn.CrossEntropyLoss()

    # init metrics
    metric_loss_train = torchmetrics.MeanMetric()
    metric_accuracy_train = torchmetrics.Accuracy(task="multiclass", num_classes=dataset.num_classes)
    metric_accuracy_val = torchmetrics.Accuracy(task="multiclass", num_classes=dataset.num_classes)

    best_model_params = None
    best_val_acc = 0.0

    # training loop
    num_epochs = 100
    epoch_bar = tqdm(list(range(num_epochs)), desc="Epochs" + " "*100)
    for epoch in epoch_bar:

        # training loop
        for data in dataloader_train:
            model.train()
            optimizer.zero_grad()
            logits = model(data.x, data.edge_index, data.batch)
            loss = criterion(logits, data.y)
            loss.backward()
            optimizer.step()

            metric_loss_train(loss)
            metric_accuracy_train(logits.argmax(dim=1), data.y)

        # evaluation loop
        for data in dataloader_val:
            model.eval()
            with torch.no_grad():
                logits = model(data.x, data.edge_index, data.batch)
                pred = logits.argmax(dim=1)
                metric_accuracy_val(pred, data.y)


        train_loss = metric_loss_train.compute().item()
        train_acc = metric_accuracy_train.compute().item()
        val_acc = metric_accuracy_val.compute().item()

        # progress bar
        epoch_bar.set_description("".join([
            f'Epoch: {epoch+1:03d} | ',
            f'Val Acc: {val_acc:.3f} | ',
            f'Train Acc: {train_acc:.3f} | ',
            f'Train Loss: {train_loss:.3f} | ',
            ]))

        # update best model
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_model_state = model.state_dict()

    best_model = model
    best_model.load_state_dict(best_model_state)

    return best_model, best_val_acc

### Download training data

**Datasets**

Comparing to other Fully-Connected Neural Networks (NNs) or Convolutional Neural Networks (CNNs),

Graph neural networks (GNNs) require have two basic datapoints: a graph structure and a feature matrix.

In [3]:
from torch_geometric.datasets import TUDataset

dataset = TUDataset(root='data/TUDataset', name='MUTAG')

### [Again] Define our model

We will use the `nn.Module` class from PyTorch to define our model.
Since having multiple datapoints we need to define a model that has multiple inputs in the `forward` function.



In [15]:
class GraphNetModel(nn.Module):

    def __init__(self, num_features, num_classes, gnnLayer=None, countingLayer=None):
        super().__init__()

        # in case no gnnLayer is provided, use GraphConv
        if not isinstance(gnnLayer, nn.Module):
            gnnLayer = gnn.SGConv

        if not isinstance(countingLayer, nn.Module):
            self.cnt1 = lambda x, edge_index: x
            add_features = 0
        else:
            self.cnt1 = countingLayer()
            add_features = self.cnt1.add_features


        # first graph covolutional layer
        self.conv1 = gnnLayer(num_features+add_features, 16)

        # TODO🚀(optional):
        # - add more conv layers

        # last graph convolutional layer
        self.convL = gnnLayer(16, 16)

        # TODO🚀(optional):
        # - in many tutorials, the stantard dropout layer
        #    on **features** is used which can help
        #    with overfitting. Simlarly, the standard
        #    batch normalization can be used as well.        
        # - `self.batchnorm = nn.BatchNorm1d(32)`

        self.dropout = nn.Dropout(0.5)

        self.linear = nn.Linear(16, num_classes)

    def forward(self, x, edge_index, batch):

        # apply counting layer
        x = self.cnt1(x, edge_index)

        # apply first layer and follow with ReLU
        x = self.conv1(x, edge_index).relu()

        # TODO🚀(optional):
        # - apply more conv layers

        x = self.convL(x, edge_index).relu()

        # this (set) pooling makes the function
        # invariant to the order of the nodes
        x = gnn.global_mean_pool(x, batch)

        # TODO🚀(optional):
        # add dropout or batchnorm of features
        x = self.dropout(x)

        # apply linear layer
        x = self.linear(x)

        return x

### [Again] Perform training and evaluation

In [16]:
model = GraphNetModel(dataset.num_features, dataset.num_classes, gnnLayer=gnn.CGConv)
model, val_accuracy = train_and_eval(model, dataset)

print("Model size:", sum(p.numel() for p in model.parameters()))
print("Validation Accuracy:", val_accuracy)

Epoch: 100 | Val Acc: 0.658 | Train Acc: 0.734 | Train Loss: 0.553 | : 100%|██████████| 100/100 [00:08<00:00, 11.74it/s]                           

Model size: 434
Validation Accuracy: 0.6904024481773376


### [Task 2] Adding Local Graph Parameters



In [23]:
class LGPCountingLayer(nn.Module):

    def __init__(self):
        super().__init__()

        # we indicate that we add 2 features in dim=1
        self.add_features = 4

    def forward(self, x, edge_index):
        nx = x.shape[0]

        # create abstraction of sparse adjacency matrix from edge_index
        A = torch.sparse_coo_tensor(edge_index, torch.ones_like(edge_index[0]).float(), size=(nx, nx))

        # also sparse identity matrix
        I = torch.sparse_coo_tensor(torch.stack([torch.arange(nx), torch.arange(nx)]), torch.ones(nx).float(), size=(nx, nx))

        # walks of length 3
        W3 = (A @ A @ A)
        # walks of length 4
        W4 = W3 @ A
        # walks of length 5
        W5 = W4 @ A
        # walks of length 6
        W6 = W5 @ A
        # walks of length 7
        W7 = W6 @ A
        # walks of length 8
        W8 = W7 @ A
        # walks of length 9
        W9 = W8 @ A
        # walks of length 10
        W10 = W9 @ A
        # walks of length 20
        W20 = W10 @ W10

        # closed walks of length 3
        C3 = (W3 * I).sum(dim=1).to_dense().view(-1, 1)
        # closed walks of length 4
        C4  = (W4 * I).sum(dim=1).to_dense().view(-1, 1)
        # closed walks of length 5
        C5  = (W5 * I).sum(dim=1).to_dense().view(-1, 1)
        # closed walks of length 6
        C6  = (W6 * I).sum(dim=1).to_dense().view(-1, 1)
        # closed walks of length 7
        C7  = (W7 * I).sum(dim=1).to_dense().view(-1, 1)
        # closed walks of length 8
        C8  = (W8 * I).sum(dim=1).to_dense().view(-1, 1)
        # closed walks of length 9
        C9  = (W9 * I).sum(dim=1).to_dense().view(-1, 1)
        # closed walks of length 10
        C10 = (W10 * I).sum(dim=1).to_dense().view(-1, 1)
        # closed walks of length 20
        C20 = (W20 * I).sum(dim=1).to_dense().view(-1, 1)



        # log scale 
        c3l, c4l = C3.log1p(), C4.log1p()
        c5l, c6l = C5.log1p(), C6.log1p()
        c7l, c8l = C7.log1p(), C8.log1p()
        c9l, c10l = C9.log1p(), C10.log1p()
        c20l = C20.log1p()

        # concatenate to standard features        
        x = torch.cat([
            x, 
            c3l, c4l, c5l, c6l, 
            c7l, c8l, c9l, c10l,
            c20l
        ], dim=1)

        return x



**train model with LGPCountingLayer:**

In [24]:
model_our1 = GraphNetModel(dataset.num_features, dataset.num_classes, gnnLayer=gnn.CGConv, countingLayer=LGPCountingLayer)
trained_model_our1, val_accuracy_our1 = train_and_eval(model_our1, dataset)

print("Model size:", sum(p.numel() for p in model_our1.parameters()))
print("Validation Accuracy:", val_accuracy_our1)

Epoch: 100 | Val Acc: 0.682 | Train Acc: 0.717 | Train Loss: 0.553 | : 100%|██████████| 100/100 [00:07<00:00, 14.16it/s]                           

Model size: 434
Validation Accuracy: 0.6968421339988708


## 2-WL-GNNs

(2023) [Two-Dimensional Weisfeiler-Lehman Graph Neural Networks for Link Prediction](https://arxiv.org/abs/2206.09567)